<a href="https://colab.research.google.com/github/Melvinchen0404/Chess_engine_v2/blob/main/sapientia_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

# Define the FEN notation
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"

# API URL
url = f"https://explorer.lichess.ovh/master?fen={fen}"

# Send the request to the API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()  # Parse the JSON response

    # Debug: Print keys in the API response
    print("Response Keys:", data.keys())

    # Debug: Check the structure of topGames
    games = data.get("topGames", [])
    if games:
        print("Example game structure:", games[0])  # Print the first game for debugging
    else:
        print("No games found.")
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")

Response Keys: dict_keys(['white', 'draws', 'black', 'moves', 'topGames', 'opening'])
Example game structure: {'uci': 'e2e4', 'id': 'boLujJkB', 'winner': 'white', 'black': {'name': 'Carlsen, M.', 'rating': 2882}, 'white': {'name': 'Caruana, F.', 'rating': 2818}, 'year': 2019, 'month': '2019-08'}


In [21]:
from gm_database import fetch_gm_data

# Define the FEN notation
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"

# Call the function and fetch data
print("Best move and analysis from Lichess GM database (human):")
fetch_gm_data(fen)

Best move and analysis from Lichess GM database (human):
Best move: e2e4
White wins: 398455
Black wins: 300394

Examples of GM games in which e2e4 was played:
Game 1: Caruana vs Carlsen (2019), Result: 1-0
Game 2: Caruana vs Carlsen (2019), Result: 1-0
Game 3: Carlsen vs Caruana (2019), Result: 1/2-1/2
Game 4: Carlsen vs Caruana (2019), Result: 1/2-1/2
Game 5: Carlsen vs Caruana (2020), Result: 1/2-1/2


In [44]:
from gm_database import fetch_gm_data
from engines import fetch_komodo_data, fetch_stockfish_data

# ANSI escape code for bold text
BOLD = "\033[1m"
RESET = "\033[0m"  # Reset formatting to normal

# Define the FEN notation
fen = "rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq e6 0 1"

# Call the function and fetch data
print(f"{BOLD}Best move and analysis from Lichess GM database (human):{RESET}")
fetch_gm_data(fen)
print()
# CAll the function and fetch data
print(f"{BOLD}Best move from Stockfish:{RESET}\n",best_move_stockfish)
print()
print(f"{BOLD}Best move from Komodo:{RESET}\n",best_move_komodo)

Best move and analysis from Lichess GM database (human):
Best move: g1f3
White wins: 78292
Black wins: 52586

Examples of GM games in which g1f3 was played:
Game 1: Carlsen vs Caruana (2019), Result: 1/2-1/2
Game 2: Caruana vs Carlsen (2019), Result: 1-0
Game 3: Caruana vs Carlsen (2020), Result: 0-1
Game 4: Carlsen vs Caruana (2020), Result: 1-0
Game 5: Carlsen vs Caruana (2020), Result: 1-0

Best move from Stockfish:
 g1f3

Best move from Komodo:
 g1f3


In [10]:
!pip install datasets python-chess

import os
import sys
import argparse
import chess
import chess.svg
from IPython.display import display, SVG, clear_output
import ipywidgets as widgets
from openings import process_moves
from gm_database import fetch_gm_data, fetch_data_from_lichess, display_top_games
from engines import fetch_komodo_data, fetch_stockfish_data
from endgame import fetch_endgame_tablebase_data

# ANSI escape code for bold text
BOLD = "\033[1m"
RESET = "\033[0m"  # Reset formatting to normal

# Create widgets for command input and submit button
input_box = widgets.Text(placeholder='Enter your move or command', description='Command:')
submit_button = widgets.Button(description="Submit")
output_area = widgets.Output()
layout_container = widgets.VBox([input_box, submit_button, output_area])

# Sequence mode flag and moves
sequence_mode = False
sequence_moves = []

# Initialize the board
board = chess.Board()

# Function to render the chessboard and display the best move, WDL, and DTZ
def render_board_and_best_move(board, best_move_gm_database=None, best_move_stockfish=None, best_move_komodo=None, wdl=None, dtz=None, scale=0.6):
    with output_area:
        clear_output(wait=True)  # Clear previous output

        # Render the chessboard
        display(SVG(chess.svg.board(board, size=400 * scale)))

        # Prepare output lines to display below the chessboard
        output_lines = []

        # Get the FEN notation of the current board
        fen = board.fen()  # Get the current FEN of the board

        # Get engine recommendations and opening data
        best_move_gm_database = fetch_gm_data(fen)
        best_move_stockfish = fetch_stockfish_data(fen, depth=15, stockfish_path="/usr/games/stockfish")
        best_move_komodo = fetch_komodo_data(fen, depth=15, komodo_path="/content/komodo3sse42")

        endgame_data = fetch_endgame_tablebase_data(fen)
        wdl = endgame_data.get("WDL", "N/A")
        dtz = endgame_data.get("DTZ", "N/A")

        if best_move_gm_database:
            output_lines.append(f"Best move and analysis from Lichess GM database: {best_move_gm_database}")
        else:
            output_lines.append("No best move or analysis available from Lichess GM database.")

        if best_move_stockfish:
            output_lines.append(f"Best move from Stockfish: {best_move_stockfish}")
        else:
            output_lines.append("No best move available from Stockfish.")

        if best_move_komodo:
            output_lines.append(f"Best move from Komodo: {best_move_komodo}")
        else:
            output_lines.append("No best move available from Komodo.")

        if wdl:
            output_lines.append(f"WDL (Win/Draw/Loss): {wdl}")
        else:
            output_lines.append("No WDL data available.")

        if dtz is not None:
            output_lines.append(f"DTZ (Distance to Zero): {dtz}")
        else:
            output_lines.append("No DTZ data available.")

        # Display the move information below the chessboard
        for line in output_lines:
            print(line)

# Function to process user commands (reset, undo, etc.)
def process_command(depth, msg):
    global sequence_mode, sequence_moves
    msg = msg.strip()

    if sequence_mode:
        sequence_moves.append(msg)
        print(f"Moves in sequence: {sequence_moves}")

        if len(sequence_moves) > 0:
            for move_uci in sequence_moves:
                process_moves(move_uci, board)
            sequence_moves = []
            sequence_mode = False
    elif msg == "quit":
        close_engine()
        sys.exit()
    elif msg == "reset":
        board.reset()
        render_board_and_best_move(board)
    elif msg == "undo":
        if len(board.move_stack) > 0:
            board.pop()
        render_board_and_best_move(board)
    elif msg == "sequence":
        print("Entering sequence mode. Enter your moves (space-separated UCI moves. For instance, 'e2e4 e7e5 g1f3'):")
        sequence_mode = True
    else:
        process_moves(msg, board)

    # Now get the FEN notation after processing the moves
    fen = board.fen()
    print(f"{BOLD}Current FEN: {fen}{RESET}")  # Debugging line

    # Get engine recommendations and opening data
    print(f"{BOLD}Best move from GM Database: {best_move_gm_database['uci']}{RESET}")  # Print the UCI move
    print(f"White wins: {best_move['white']}")
    print(f"Black wins: {best_move['black']}")
    print(f"{BOLD}Best move from Stockfish: {best_move_stockfish}{RESET}")  # Debugging line
    print(f"{BOLD}Best move from Komodo: {best_move_komodo}{RESET}")  # Debugging line

    endgame_data = fetch_endgame_tablebase_data(fen)
    wdl = endgame_data.get("WDL", "N/A")
    dtz = endgame_data.get("DTZ", "N/A")

    render_board_and_best_move(board, best_move_gm_database, best_move_komodo, best_move_stockfish, wdl, dtz)

# Function to handle button clicks for submitting commands
def on_button_click(b):
    msg = input_box.value
    process_command(get_depth(), msg)
    input_box.value = ""  # Clear the input box

def get_depth():
    parser = argparse.ArgumentParser()
    parser.add_argument("--depth", default=3, type=int, help="provide an integer (default: 3)")
    args, unknown = parser.parse_known_args([arg for arg in sys.argv if arg.startswith("--")])
    return max(1, args.depth)

# Function to display the interactive command line and board
def talk():
    submit_button.on_click(on_button_click)  # Bind button click event
    display(layout_container)  # Display command input, button, and output area
    render_board_and_best_move(board)  # Render the initial board and recommendations

if __name__ == "__main__":
    talk()


In [6]:
!pip install datasets python-chess

import os
import sys
import argparse
import chess
import chess.svg
from IPython.display import display, SVG, clear_output
import ipywidgets as widgets
from openings import process_moves
from gm_database import fetch_gm_data, fetch_data_from_lichess, display_top_games
from engines import fetch_komodo_data, fetch_stockfish_data
from endgame import fetch_endgame_tablebase_data

# ANSI escape code for bold text
BOLD = "\033[1m"
RESET = "\033[0m"  # Reset formatting to normal

# Create widgets for command input and submit button
input_box = widgets.Text(placeholder='Enter your move or command', description='Command:')
submit_button = widgets.Button(description="Submit")
output_area = widgets.Output()
layout_container = widgets.VBox([input_box, submit_button, output_area])

# Sequence mode flag and moves
sequence_mode = False
sequence_moves = []

# Initialize the board
board = chess.Board()

# Function to render the chessboard and display the best move, WDL, and DTZ
def render_board_and_best_move(board, best_move_gm_database=None, best_move_stockfish=None, best_move_komodo=None, wdl=None, dtz=None, scale=0.6):
    with output_area:
        clear_output(wait=True)  # Clear previous output

        # Render the chessboard
        display(SVG(chess.svg.board(board, size=400 * scale)))

        # Prepare output lines to display below the chessboard
        output_lines = []

        # Get engine recommendations and opening data
        fen = board.fen()  # Get the current FEN of the board
        best_move_gm_database = fetch_gm_data(fen)
        best_move_stockfish = fetch_stockfish_data(fen, depth=15, stockfish_path="/usr/games/stockfish")
        best_move_komodo = fetch_komodo_data(fen, depth=15, komodo_path="/content/komodo3sse42")

        endgame_data = fetch_endgame_tablebase_data(fen)
        wdl = endgame_data.get("WDL", "N/A")
        dtz = endgame_data.get("DTZ", "N/A")

        if best_move_gm_database:
            output_lines.append(f"{BOLD}Best move from GM Database: {best_move_gm_database['uci']}\n White wins: {best_move['white']} \n Black wins: {best_move['black']}{RESET}")
        else:
            output_lines.append("No best move or analysis available from Lichess GM database.")

        if best_move_stockfish:
            output_lines.append(f"{BOLD}Best move from Stockfish: {best_move_stockfish}{RESET}")  # Debugging line
        else:
            output_lines.append("No best move available from Stockfish.")

        if best_move_komodo:
            output_lines.append(f"{BOLD}Best move from Komodo: {best_move_komodo}{RESET}")
        else:
            output_lines.append("No best move available from Komodo.")

        if wdl:
            output_lines.append(f"WDL (Win/Draw/Loss): {wdl}")
        else:
            output_lines.append("No WDL data available.")

        if dtz is not None:
            output_lines.append(f"DTZ (Distance to Zero): {dtz}")
        else:
            output_lines.append("No DTZ data available.")

        # Display the move information below the chessboard
        for line in output_lines:
            print(line)

# Function to process user commands (reset, undo, etc.)
def process_command(depth, msg):
    global sequence_mode, sequence_moves
    msg = msg.strip()

    if sequence_mode:
        sequence_moves.append(msg)
        print(f"Moves in sequence: {sequence_moves}")

        if len(sequence_moves) > 0:
            for move_uci in sequence_moves:
                process_moves(move_uci, board)
            sequence_moves = []
            sequence_mode = False
    elif msg == "quit":
        close_engine()
        sys.exit()
    elif msg == "reset":
        board.reset()
        render_board_and_best_move(board)
    elif msg == "undo":
        if len(board.move_stack) > 0:
            board.pop()
        render_board_and_best_move(board)
    elif msg == "sequence":
        print("Entering sequence mode. Enter your moves (space-separated UCI moves. For instance, 'e2e4 e7e5 g1f3'):")
        sequence_mode = True
    else:
        process_moves(msg, board)

    # Now get the FEN notation after processing the moves
    fen = board.fen()
    print(f"{BOLD}Current FEN: {fen}{RESET}")  # Debugging line

    render_board_and_best_move(board, best_move_gm_database, best_move_komodo, best_move_stockfish, wdl, dtz)

# Function to handle button clicks for submitting commands
def on_button_click(b):
    msg = input_box.value
    process_command(get_depth(), msg)
    input_box.value = ""  # Clear the input box

def get_depth():
    parser = argparse.ArgumentParser()
    parser.add_argument("--depth", default=3, type=int, help="provide an integer (default: 3)")
    args, unknown = parser.parse_known_args([arg for arg in sys.argv if arg.startswith("--")])
    return max(1, args.depth)

# Function to display the interactive command line and board
def talk():
    submit_button.on_click(on_button_click)  # Bind button click event
    display(layout_container)  # Display command input, button, and output area
    render_board_and_best_move(board)  # Render the initial board and recommendations

if __name__ == "__main__":
    talk()


Current FEN: rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1


NameError: name 'best_move_gm_database' is not defined